# Using Existing GPU code inside Matlab

There are many ways to incorporate GPU code into your Matlab programs. We've already seen how to use gpuArray to perform Fourier transforms, random number generation, and several point-wise operations on the GPU. However, should we want to do something a little more complex than a point-wise operation, we are limited to the methods supported by gpuArray.

Fortunately, there is something we can do about this! When we have an existing CUDA kernel, we can call it from Matlab
using `parallel.gpu.CUDAKernel()` constructor. 

Alternatively, you could write a MexFunction (in C or C++) that calls the GPU functions from C/C++ and call that MexFunction from Matlab. There could be some benefits to using mexfiles. It's particularly useful when you already have a lot of code in C that you want call from Matlab. The disadvantage is that Mexfiles have to be compiled and are not very portable. In most situations, however, using the CUDAKernel object should be sufficient.

Let's look at how to use the `parallel.gpu.CUDAKernel()` constructor:

In [ ]:
help parallel.gpu.CUDAKernel

As you can see there are multiple ways to construct CUDAKernels. We'll look at some examples, say we have a simple CUDA kernel to add two vectors stored in a file called `vector_add.cu`, like this:

In [ ]:
kernel_string = sprintf([ ...
'__global__ void vector_add(float *c, const float *a, const float *b, const int n) {\n' ...
'    int i = threadIdx.x;  /* <--- Oops! something is not right here! */ \n' ...
'    if (i<n) {\n' ...
'        c[i] = a[i] + b[i];\n' ...
'    }\n' ...
'}\n']);

fid = fopen('vector_add.cu','wt');
fprintf(fid, kernel_string);
fclose(fid);

disp(kernel_string);

The first compile step for any CUDA kernel is from CUDA source code to PTX code. You can compile to ptx using the `-ptx` option of nvcc. You can do this directly by executing the following cell:

In [ ]:
!nvcc -arch=sm_60 -ptx vector_add.cu

Now that we have a PTX file for our vector_add kernel we can create a CUDAKernel object:

In [ ]:
vector_add = parallel.gpu.CUDAKernel('vector_add.ptx', 'vector_add.cu', 'vector_add')

When we execute the above cell Matlab is kind enough to print some information about our kernel. First of all we can see that Matlab has read the source code of our vector_add.cu file to learn that there is one output argument (the `float *c`) and four right hand side arguments. The other arguments are regarded as input-only because we have indicated this using `const` in the datatype.

We can also see that the kernel currently uses ``ThreadBlockSize: [1, 1, 1]`` and ``GridSize: [1 1 1]``. This means that our vector_add kernel will be executed by only one thread block with only one thread in it. This is something that we have to change before we can run the kernel on a larger array. Let's create some input data and look at how we can set the grid and thread block dimensions properly.

In [ ]:
n = 10000000;

a = randn( n, 'single', 'gpuArray');
b = randn( n, 'single', 'gpuArray');
c = zeros( n, 'single', 'gpuArray');

%set thread block size as 512
vector_add.ThreadBlockSize = [512 1 1];

%compute the grid size
vector_add.GridSize = [ceil(n/512) 1 1];

We've created three gpuArrays, two filled with random numbers and an array filled with zeros. Also note, that we explicitly ask `randn` and `zeros` for arrays filled with single precision floating-point values. This is because our CUDA kernel expects `float *` as in and output.

We've arbitrarly selected a thread block dimension of (512,1,1). For this particular kernel it won't matter much what thread block dimensions you pick for the x-dimension. For some kernels, however, have their thread block dimensions fixed at compile time and will only produce correct results when executed with the right number of threads per block.

The grid dimensions are computed from the size of our problem and the size of thread block. This is because we want to create a sufficient number of thread blocks, such that when all thread blocks are done executing we have processed the entire input. As you can see the grid dimension is rounded up, this is because we the thread block size may not be
a divisor of our problem size and we also want to create a thread block for last bit of our array. This could mean that we create more threads than strictly necessary, which in turn could lead to errors when thread attempt to access memory out of array bounds. That is why there is an if-statement in our CUDA kernel that checks whether the global index of thread is smaller than n.

Now we are ready to call the vector_add kernel.

In [ ]:
%call the vector_add kernel
c = feval(vector_add,c,a,b,n);

%compute the same in Matlab
answer = gather(a) + gather(b);

if all(abs(gather(c)-answer) < 1e8)
    disp('TEST PASSED!')
else
    disp('TEST FAILED!')
end
    

The above code called our CUDAKernel using `feval`. The first argument should be the CUDAKernel object, the rest of the arguments should match the argument list of the kernel. After calling the kernel the result is stored in c.

To test whether our CUDAKernel executed correctly we also perform the same computation in Matlab. To do this we first have to `gather()` the gpuArrays a and b from GPU memory to the Matlab workspace. In case you are unsure of the type of your variables (whether they are gpuArrays or not, for example) you can use the following command:

In [ ]:
whos

The output of whos shows which arrays are on the GPU and which are part of the Matlab workspace. 

Finally, we compare the two outputs to verify that there are no big differences in the results. Note that floating-point values should not be compared directly as there can always be small rounding errors that do not necessarily indicate that something went wrong.

If our little test above has printed "TEST FAILED", you should look again at the CUDA kernel code in the second code cell of this notebook and see if you can find out what is wrong with it! Don't forget to re-run the 3rd and 4th cells to recompile the kernel and reload it into Matlab!